In [1]:
import sys
sys.path.append("..")

In [2]:
import re
from collections import defaultdict
from itertools import combinations

from resources.utils import get_puzzle_input

### Part 1

It's no use; your navigation system simply isn't capable of providing walking directions in the arctic circle, and certainly not in 1018.

The Elves suggest an alternative. In times like these, North Pole rescue operations will arrange points of light in the sky to guide missing Elves back to base. Unfortunately, the message is easy to miss: the points move slowly enough that it takes hours to align them, but have so much momentum that they only stay aligned for a second. If you blink at the wrong time, it might be hours before another message appears.

You can see these points of light floating in the distance, and record their position in the sky and their velocity, the relative change in position per second (your puzzle input). The coordinates are all given from your perspective; given enough time, those positions and velocities will move the points into a cohesive message!

Rather than wait, you decide to fast-forward the process and calculate what the points will eventually spell.

For example, suppose you note the following points:

```
position=< 9,  1> velocity=< 0,  2>
position=< 7,  0> velocity=<-1,  0>
position=< 3, -2> velocity=<-1,  1>
position=< 6, 10> velocity=<-2, -1>
position=< 2, -4> velocity=< 2,  2>
position=<-6, 10> velocity=< 2, -2>
position=< 1,  8> velocity=< 1, -1>
position=< 1,  7> velocity=< 1,  0>
position=<-3, 11> velocity=< 1, -2>
position=< 7,  6> velocity=<-1, -1>
position=<-2,  3> velocity=< 1,  0>
position=<-4,  3> velocity=< 2,  0>
position=<10, -3> velocity=<-1,  1>
position=< 5, 11> velocity=< 1, -2>
position=< 4,  7> velocity=< 0, -1>
position=< 8, -2> velocity=< 0,  1>
position=<15,  0> velocity=<-2,  0>
position=< 1,  6> velocity=< 1,  0>
position=< 8,  9> velocity=< 0, -1>
position=< 3,  3> velocity=<-1,  1>
position=< 0,  5> velocity=< 0, -1>
position=<-2,  2> velocity=< 2,  0>
position=< 5, -2> velocity=< 1,  2>
position=< 1,  4> velocity=< 2,  1>
position=<-2,  7> velocity=< 2, -2>
position=< 3,  6> velocity=<-1, -1>
position=< 5,  0> velocity=< 1,  0>
position=<-6,  0> velocity=< 2,  0>
position=< 5,  9> velocity=< 1, -2>
position=<14,  7> velocity=<-2,  0>
position=<-3,  6> velocity=< 2, -1>
```

Each line represents one point. Positions are given as <X, Y> pairs: X represents how far left (negative) or right (positive) the point appears, while Y represents how far up (negative) or down (positive) the point appears.

At 0 seconds, each point has the position given. Each second, each point's velocity is added to its position. So, a point with velocity <1, -2> is moving to the right, but is moving upward twice as quickly. If this point's initial position were <3, 9>, after 3 seconds, its position would become <6, 3>.

Over time, the points listed above would move like this:

```
Initially:
........#.............
................#.....
.........#.#..#.......
......................
#..........#.#.......#
...............#......
....#.................
..#.#....#............
.......#..............
......#...............
...#...#.#...#........
....#..#..#.........#.
.......#..............
...........#..#.......
#...........#.........
...#.......#..........

After 1 second:
......................
......................
..........#....#......
........#.....#.......
..#.........#......#..
......................
......#...............
....##.........#......
......#.#.............
.....##.##..#.........
........#.#...........
........#...#.....#...
..#...........#.......
....#.....#.#.........
......................
......................

After 2 seconds:
......................
......................
......................
..............#.......
....#..#...####..#....
......................
........#....#........
......#.#.............
.......#...#..........
.......#..#..#.#......
....#....#.#..........
.....#...#...##.#.....
........#.............
......................
......................
......................

After 3 seconds:
......................
......................
......................
......................
......#...#..###......
......#...#...#.......
......#...#...#.......
......#####...#.......
......#...#...#.......
......#...#...#.......
......#...#...#.......
......#...#..###......
......................
......................
......................
......................

After 4 seconds:
......................
......................
......................
............#.........
........##...#.#......
......#.....#..#......
.....#..##.##.#.......
.......##.#....#......
...........#....#.....
..............#.......
....#......#...#......
.....#.....##.........
...............#......
...............#......
......................
......................
```

After 3 seconds, the message appeared briefly: HI. Of course, your message will be much longer and will take many more seconds to appear.

What message will eventually appear in the sky?

In [3]:
test_input = """
position=< 9,  1> velocity=< 0,  2>
position=< 7,  0> velocity=<-1,  0>
position=< 3, -2> velocity=<-1,  1>
position=< 6, 10> velocity=<-2, -1>
position=< 2, -4> velocity=< 2,  2>
position=<-6, 10> velocity=< 2, -2>
position=< 1,  8> velocity=< 1, -1>
position=< 1,  7> velocity=< 1,  0>
position=<-3, 11> velocity=< 1, -2>
position=< 7,  6> velocity=<-1, -1>
position=<-2,  3> velocity=< 1,  0>
position=<-4,  3> velocity=< 2,  0>
position=<10, -3> velocity=<-1,  1>
position=< 5, 11> velocity=< 1, -2>
position=< 4,  7> velocity=< 0, -1>
position=< 8, -2> velocity=< 0,  1>
position=<15,  0> velocity=<-2,  0>
position=< 1,  6> velocity=< 1,  0>
position=< 8,  9> velocity=< 0, -1>
position=< 3,  3> velocity=<-1,  1>
position=< 0,  5> velocity=< 0, -1>
position=<-2,  2> velocity=< 2,  0>
position=< 5, -2> velocity=< 1,  2>
position=< 1,  4> velocity=< 2,  1>
position=<-2,  7> velocity=< 2, -2>
position=< 3,  6> velocity=<-1, -1>
position=< 5,  0> velocity=< 1,  0>
position=<-6,  0> velocity=< 2,  0>
position=< 5,  9> velocity=< 1, -2>
position=<14,  7> velocity=<-2,  0>
position=<-3,  6> velocity=< 2, -1>
"""

In [4]:
input_re  = re.compile(r'position=<\s*([0-9-]+),\s*([0-9-]+)> velocity=<\s*([0-9-]+),\s*([0-9-]+)>')

In [5]:
class Star:
    def __init__(self, x, y, vel_x, vel_y):
        self.x = x
        self.y = y
        self.vel_x = vel_x
        self.vel_y = vel_y
        
    def move(self):
        self.x += self.vel_x
        self.y += self.vel_y
    
    def is_neighbour(self, other_star):
        return abs(self.x - other_star.x) <= 1 and abs(self.y - other_star.y) <= 1
    
    def __key(self):
        return (self.x, self.y, self.vel_x, self.vel_y)

    def __hash__(self):
        return hash(self.__key())

    def __eq__(self, other):
        return isinstance(self, type(other)) and self.__key() == other.__key()
    
    def __repr__(self):
        return "<{},{}> <{},{}>".format(
            self.x,
            self.y,
            self.vel_x,
            self.vel_y
        )

In [6]:
def parse_stars(input):
    stars = []
    for line in input:
        match = input_re.match(line)
        if match:
            args = [int(x) for x in match.groups()]
            stars.append(Star(*args))

    return stars

In [7]:
test_stars = parse_stars(test_input.split('\n'))

In [8]:
assert Star(1, 2, 1, 1).is_neighbour(Star(2, 2, 1, 1))

In [9]:
class Sky():
    def __init__(self, stars):
        self.stars = stars
    
    def move(self):
        for star in self.stars:
            star.move()

    @property
    def aligned(self):
        unaligned = set(self.stars)
        for star_1 in self.stars:
            if star_1 not in unaligned:
                continue
            
            aligned = False
            for star_2 in self.stars:
                if star_1 != star_2 and star_1.is_neighbour(star_2):
                    unaligned.remove(star_1)
                    if star_2 in unaligned:
                        unaligned.remove(star_2)
                    aligned = True
                    break
            
            if not aligned:
                return False
        
        return True
    
    @property
    def grid_corners(self):
        min_x = None
        max_x = None
        min_y = None
        max_y = None
        
        for star in self.stars:
            if max_x is None or star.x > max_x:
                max_x = star.x
            if min_x is None or star.x < min_x:
                min_x = star.x
            if max_y is None or star.y > max_y:
                max_y = star.y
            if min_y is None or star.y < min_y:
                min_y = star.y
        
        return min_x, min_y, max_x, max_y
            
            
    def __repr__(self):
        min_x, min_y, max_x, max_y = self.grid_corners

        grid = defaultdict(lambda *_: defaultdict(lambda *_: '.'))
        
        for star in self.stars:
            grid[star.x - min_x][star.y - min_y] = '#'
        
        output = []
        for x in range(max_x - min_x + 2):
            line = []
            for y in range(max_y - min_y + 2):
                line.append(grid[x][y])
            output.append(''.join(line))
        
        return '\n'.join(output)

In [10]:
aligned_sky = Sky([
    Star(1, 2, 1, 1),
    Star(2, 2, 1, 1)
])
assert aligned_sky.aligned

In [11]:
def solve(stars):
    sky = Sky(stars)
    x = 0
    while not sky.aligned:
        sky.move()
        x += 1
    print(x)
    
    return sky

In [12]:
solve(test_stars)

3


########.
...#.....
...#.....
...#.....
########.
.........
.........
#......#.
########.
#......#.
.........

In [13]:
puzzle_input = get_puzzle_input('/tmp/day_10.txt')
puzzle_stars = parse_stars(puzzle_input)

In [14]:
solve(puzzle_stars)

10375


##########.
#...#......
#...#......
#...#......
#...#......
.###.......
...........
...........
##########.
#...#......
#...#......
#...#......
#...#......
.###.......
...........
...........
##########.
.##........
...##......
.....##....
.......##..
##########.
...........
...........
.......##..
.........#.
.........#.
#........#.
#########..
#..........
...........
...........
##########.
#...#....#.
#...#....#.
#...#....#.
#...#....#.
#........#.
...........
...........
##########.
#...#....#.
#...#....#.
#...#....#.
#...#....#.
#........#.
...........
...........
##########.
.##........
...##......
.....##....
.......##..
##########.
...........
...........
##########.
....#......
....#......
....#......
....#......
##########.
...........

In [15]:
# PPNJEENH

### Part 2

Good thing you didn't have to wait, because that would have taken a long time - much longer than the 3 seconds in the example above.

Impressed by your sub-hour communication capabilities, the Elves are curious: exactly how many seconds would they have needed to wait for that message to appear?